In [ ]:
# for running Ollama in Google colab
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

Once the xterm is open, we can proceed with the installation of Ollama by running the commented commands in the xterm:

In [ ]:
%xterm
# run cmd below in xterm
 # curl https://ollama.ai/install.sh | sh
 # ollama serve

In [ ]:
%pip install urban-worm --upgrade

Import dataset class

In [1]:
from urbanworm.inference import llama
from urbanworm.dataset import GeoTaggedData

In [6]:
# get building footprints from OSM
# Initiate the constructor
gtd = GeoTaggedData()
# Define the area of interest using a bounding box (bbox)
bbox = (-83.208003,42.374646,-83.206608,42.375328) # in Detroit, USA
# we can just get house with no more than 200 square meter (single family houses with garage excluded)
gtd.getBuildings(bbox, min_area=60, max_area=200)
gtd.units.plot()

'5 buildings found in the bounding box.'

In [ ]:
gtd.get_svi_from_locations(key = "mapillary_key", # api key
                           distance = 30,       # only search for available street view with 30 meters from the house location
                           pano = True,         # only search for 360-degree street view images
                           reoriented = True,   # reorient and crop the street view images to make them only frame the house at the center of scene
                           multi_num = 3,       # return three closest street views from the house location
                           fov = 80,            # The field of view in degrees for the reoriented images
                           interval = 2,        # The interval between each street view (i.g, `interval = 2` means there should be two available images between two collected images)
                           year = (2024, 2025), # only search for images captured between 2024 and 2025
                           time_of_day = 'day'  # only search for images captured during the daytime
                           )

# indicate that the street view images wil be used
gtd.set_images('svi')
# pass the dataset to the inference constructor
data = llama.InferenceOllama(geo_tagged_data=gtd)

In [ ]:
from typing import Literal
multiple_choice_format = {
    "questions": (str, ...),
    "answer": (Literal['occupied', 'unoccupied'], ...),
    "explanation": (str, ...),
}

In [8]:
prompt = '''
    Question: Does this house look occupied if it is not a vacant lot?

    **An occupied house means that the house is not abandoned and
    some people may live in this house even if there is not people outside**
'''
data.batch_inference(prompt=prompt)